In [65]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [66]:
#df = pd.read_csv("Fallvilt_beriket_med_vær.csv",sep=";", low_memory=False)
df = pd.read_csv('Fallvilt_månedsberiket.csv', sep=";", low_memory=False)

In [67]:
df=df[df['Art'].isin(['Elg', 'Hjort', 'Rådyr'])].copy()  
#df=df[df['Art'].isin(['Hjort'])].copy()  
df=df[df['vegkategori'].isin(['E','F','K'])].copy()
df=df[df['År']==2025].copy()

In [68]:
df.columns

Index(['Dato', 'År', 'Kommune', 'Stedfesting', 'Art', 'Kjønn', 'Alder',
       'Årsak', 'Utfall', 'Merkelappnummer', 'Fallvilt-ID', 'UTM33 øst',
       'UTM33 nord', 'vegsystemreferanse.kortform', 'vegkategori', 'fase',
       'vegnr', 'strekning', 'delstrekning', 'arm', 'adskilte_løp',
       'trafikantgruppe', 'retning', 'meter', 'veglenkesekvensid',
       'relativPosisjon', 'veglenkesekvens.kortform', 'geometri.wkt',
       'geometri.srid', 'kommune (treff)', 'avstand_vegnettet_m',
       'Vegobjekt_540_id', 'ÅDT, total', 'Vegobjekt_105_id', 'Fartsgrense',
       'Vegobjekt_540_lengde', 'snow_depth', 'max_temperature',
       'min_temperature', 'mean_temperature', 'total_precipitation',
       'max_wind_speed', 'mean_wind_speed', 'max_wind_gust',
       'precipitation_type', 'weather_station_id', 'Måned',
       'monthly_snow_depth', 'monthly_mean_temperature',
       'monthly_mean_wind_speed'],
      dtype='object')

In [69]:
df=df[['Vegobjekt_105_id','Dato','Art','ÅDT, total','Vegobjekt_540_lengde','adskilte_løp', 'monthly_snow_depth']].dropna().copy()

In [70]:
df["Dato"] = pd.to_datetime(df["Dato"]).copy()

In [71]:
def maaned_til_arstid(m):
    if m in [12, 1, 2]:
        return "Vinter"
    elif m in [3, 4, 5]:
        return "Vår"
    elif m in [6, 7, 8]:
        return "Sommar"
    else:
        return "Haust"

In [72]:
df["årstid"] = df["Dato"].dt.month.apply(maaned_til_arstid)
df["årstid"] = df["årstid"].astype("category").copy()

In [73]:
df

,Vegobjekt_105_id,Dato,Art,"ÅDT, total",Vegobjekt_540_lengde,adskilte_løp,monthly_snow_depth,årstid
231,135094429.0,2025-12-31,Elg,5630.0,9450.411,Nei,7.700,Vinter
234,85311783.0,2025-12-31,Rådyr,640.0,3192.223,Nei,0.000,Vinter
237,86974451.0,2025-12-30,Rådyr,700.0,17818.619,Nei,2.800,Vinter
241,85310248.0,2025-12-30,Rådyr,300.0,3913.985,Nei,7.000,Vinter
245,86975655.0,2025-12-30,Rådyr,697.0,28148.139,Nei,0.000,Vinter
...,...,...,...,...,...,...,...,...
2934,843859022.0,2025-01-01,Elg,1700.0,3122.634,Nei,20.000,Vinter
2935,728125448.0,2025-01-01,Elg,1600.0,11320.552,Nei,20.000,Vinter
2936,86975513.0,2025-01-01,Elg,800.0,12995.376,Nei,30.500,Vinter
2937,86973808.0,2025-01-01,Elg,1210.0,2278.729,Nei,36.125,Vinter


In [74]:
df["snokategori"] = pd.cut(
    df["monthly_snow_depth"],
    bins=[-0.1, 0, 10, 40, float("inf")],
    labels=["Ingen snø", "Litt snø", "Vesentleg snø", "Mye snø"]
)

In [75]:
df["snø"] = (df["monthly_snow_depth"] > 0).astype(int)
df["snø"] = df["snø"].map({0: "Ikkje snø", 1: "Snø"})
df["snø"] = df["snø"].astype("category")
df["Art"] = df["Art"].astype("category")
#df["adskilte_løp"] = df["adskilte_løp"].astype("category")

In [76]:
df=df[['Vegobjekt_105_id','ÅDT, total','Vegobjekt_540_lengde','Art','snø','årstid', 'snokategori']].copy()

In [77]:
df

,Vegobjekt_105_id,"ÅDT, total",Vegobjekt_540_lengde,Art,snø,årstid,snokategori
231,135094429.0,5630.0,9450.411,Elg,Snø,Vinter,Litt snø
234,85311783.0,640.0,3192.223,Rådyr,Ikkje snø,Vinter,Ingen snø
237,86974451.0,700.0,17818.619,Rådyr,Snø,Vinter,Litt snø
241,85310248.0,300.0,3913.985,Rådyr,Snø,Vinter,Litt snø
245,86975655.0,697.0,28148.139,Rådyr,Ikkje snø,Vinter,Ingen snø
...,...,...,...,...,...,...,...
2934,843859022.0,1700.0,3122.634,Elg,Snø,Vinter,Vesentleg snø
2935,728125448.0,1600.0,11320.552,Elg,Snø,Vinter,Vesentleg snø
2936,86975513.0,800.0,12995.376,Elg,Snø,Vinter,Vesentleg snø
2937,86973808.0,1210.0,2278.729,Elg,Snø,Vinter,Vesentleg snø


In [78]:
df["eksponering"] = (
    df["ÅDT, total"]
    * 365
    * df["Vegobjekt_540_lengde"]
    / 100_000
)

df["log_eksponering"] = np.log(df["eksponering"])


In [79]:
df

,Vegobjekt_105_id,"ÅDT, total",Vegobjekt_540_lengde,Art,snø,årstid,snokategori,eksponering,log_eksponering
231,135094429.0,5630.0,9450.411,Elg,Snø,Vinter,Litt snø,194201.220844,12.176650
234,85311783.0,640.0,3192.223,Rådyr,Ikkje snø,Vinter,Ingen snø,7457.032928,8.916913
237,86974451.0,700.0,17818.619,Rådyr,Snø,Vinter,Litt snø,45526.571545,10.726051
241,85310248.0,300.0,3913.985,Rådyr,Snø,Vinter,Litt snø,4285.813575,8.363066
245,86975655.0,697.0,28148.139,Rådyr,Ikkje snø,Vinter,Ingen snø,71610.273023,11.178994
...,...,...,...,...,...,...,...,...,...
2934,843859022.0,1700.0,3122.634,Elg,Snø,Vinter,Vesentleg snø,19375.943970,9.871788
2935,728125448.0,1600.0,11320.552,Elg,Snø,Vinter,Vesentleg snø,66112.023680,11.099106
2936,86975513.0,800.0,12995.376,Elg,Snø,Vinter,Vesentleg snø,37946.497920,10.543932
2937,86973808.0,1210.0,2278.729,Elg,Snø,Vinter,Vesentleg snø,10064.006628,9.216721


In [80]:
df_agg = (
    df
    .groupby(
        ["Vegobjekt_105_id","årstid", "snø", 'snokategori'],
        observed=True,      # fjern FutureWarning
        as_index=False
    )
    .agg(
        antall_kollisjoner=("Vegobjekt_105_id", "count"),
        log_eksponering=("log_eksponering", "first")
    )
)

In [81]:
df_agg

,Vegobjekt_105_id,årstid,snø,snokategori,antall_kollisjoner,log_eksponering
0,7.870534e+07,Vinter,Ikkje snø,Ingen snø,1,5.525863
1,7.870534e+07,Haust,Ikkje snø,Ingen snø,1,4.181336
2,7.870534e+07,Haust,Snø,Litt snø,2,4.181336
3,7.870594e+07,Vår,Snø,Litt snø,1,2.169558
4,7.870722e+07,Vinter,Snø,Vesentleg snø,1,5.884972
...,...,...,...,...,...,...
968,1.026251e+09,Vinter,Snø,Litt snø,1,11.208498
969,1.026251e+09,Haust,Snø,Litt snø,1,11.117852
970,1.026251e+09,Vinter,Snø,Litt snø,1,11.117852
971,1.026251e+09,Vinter,Snø,Vesentleg snø,1,11.117852


In [82]:
# 1. Summen skal stemme
df_agg["antall_kollisjoner"].sum() == len(df)

True

In [83]:
# 2. Ingen NaN
df_agg.isna().sum()

Vegobjekt_105_id      0
årstid                0
snø                   0
snokategori           0
antall_kollisjoner    0
log_eksponering       0
dtype: int64

In [84]:
# 3. Fornuftige gruppestorleikar
df_agg["antall_kollisjoner"].describe()

count    973.000000
mean       1.355601
std        0.835775
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max       10.000000
Name: antall_kollisjoner, dtype: float64

In [85]:
#f_agg=df_agg[df_agg['antall_kollisjoner']<10].copy()

In [86]:
df_agg["antall_kollisjoner"].max()

10

In [98]:
model_nb = smf.glm(
    formula="antall_kollisjoner ~ C(årstid)+C(snokategori)",
    data=df_agg,
    family=sm.families.NegativeBinomial(),
    offset=df_agg["log_eksponering"]
).fit()

print(model_nb.summary())



                 Generalized Linear Model Regression Results                  
Dep. Variable:     antall_kollisjoner   No. Observations:                  973
Model:                            GLM   Df Residuals:                      966
Model Family:        NegativeBinomial   Df Model:                            6
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1888.9
Date:                Wed, 04 Feb 2026   Deviance:                       776.35
Time:                        10:07:12   Pearson chi2:                 1.19e+04
No. Iterations:                    10   Pseudo R-squ. (CS):            0.01064
Covariance Type:            nonrobust                                         
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

C:\Users\jens.nilsen\python\WPy64-31241\python-3.12.4.amd64\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [88]:
model_nb = smf.glm(
    formula="antall_kollisjoner ~ C(årstid)",
    data=df_agg,
    family=sm.families.Poisson(),
    offset=df_agg["log_eksponering"]
).fit()

print(model_nb.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:     antall_kollisjoner   No. Observations:                  973
Model:                            GLM   Df Residuals:                      969
Model Family:                 Poisson   Df Model:                            3
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1874.7
Date:                Wed, 04 Feb 2026   Deviance:                       1627.8
Time:                        10:06:32   Pearson chi2:                 2.15e+04
No. Iterations:                     6   Pseudo R-squ. (CS):           0.009073
Covariance Type:            nonrobust                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept             -10.3284    

In [89]:
import numpy as np

def lag_arstidsjustering(model):
    """
    Lag justeringsfaktorar for årstid frå ein statsmodels GLM (NB / Poisson).

    Referansekategori får faktor 1.0.
    """
    params = model.params

    # Finn alle årstids-koeffisientar
    arstid_params = {
        k: v for k, v in params.items()
        if k.startswith("C(årstid)")
    }

    # Referanse (den som ikkje er i params)
    arstid_justering = {"Haust": 1.0}

    # Legg til dei estimerte årstidene
    for k, beta in arstid_params.items():
        # Trekk ut årstidsnamn, t.d. C(årstid)[T.Sommar] -> Sommar
        arstid = k.split("[T.")[1].rstrip("]")
        arstid_justering[arstid] = float(np.exp(beta))

    return arstid_justering


In [90]:
ARSTID_JUSTERING = lag_arstidsjustering(model_nb)


In [91]:
ARSTID_JUSTERING

{'Haust': 1.0,
 'Sommar': 0.9082757342540899,
 'Vinter': 0.9745952288063087,
 'Vår': 0.7962495038056134}

In [92]:
(
    df_agg
    .groupby("årstid", observed=True)["antall_kollisjoner"]
    .sum()
    .sort_values(ascending=False)
)


årstid
Vinter    488
Haust     442
Vår       219
Sommar    170
Name: antall_kollisjoner, dtype: int64

In [93]:
(
    df
    .groupby("årstid", observed=True)
    .size()
    .sort_values(ascending=False)
)


årstid
Vinter    488
Haust     442
Vår       219
Sommar    170
dtype: int64

In [94]:
(
    df_agg
    .groupby("årstid", observed=True)["antall_kollisjoner"]
    .sum()
    .pipe(lambda s: s / s.sum())
)


årstid
Haust     0.335102
Sommar    0.128886
Vinter    0.369977
Vår       0.166035
Name: antall_kollisjoner, dtype: float64

In [95]:
(
    df
    .groupby("årstid", observed=True)
    .apply(
        lambda g: g.shape[0] / g["eksponering"].sum()
    )
    .sort_values(ascending=False)
)


C:\Users\jens.nilsen\AppData\Local\Temp\ipykernel_24832\484447156.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


årstid
Sommar    0.000023
Haust     0.000021
Vinter    0.000019
Vår       0.000019
dtype: float64